In [ ]:
import matplotlib.dates as mdates
import matplotlib.pylab as plt
from matplotlib.ticker import MaxNLocator

from src.visualisation.visualize import apply_mpl_settings

apply_mpl_settings()
import datetime
import json
from collections import defaultdict
from pathlib import Path

import numpy as np
import scipy.stats as stats
import pandas as pd

from src.data.analysis import Circle, get_mean_std, get_timestamp
from src.visualisation.visualize import apply_defaults, apply_grid

# Dataset

In [ ]:
interim_path = Path('..', '..', 'data', 'interim', 'foils')
raw_path = Path('..', '..', 'data', 'raw', 'foils')

In [ ]:
ref_dataset = '2020_10_12_Co60'
ref_dose_Gy = 60.
ref_dataset_path = Path(interim_path, ref_dataset)
list_of_ref_datasets = []
if ref_dataset_path.exists():
    list_of_ref_datasets = sorted((entry.name for entry in ref_dataset_path.iterdir() if entry.name.isdigit()), key=lambda s: int(s))
' '.join(list_of_ref_datasets)

In [ ]:
dataset = '2022_11_17_bp'
#dataset = '2022_11_18_sobp'
#dataset = '2022_11_03_Co60'
dataset_path = Path(interim_path, dataset)
list_of_datasets = []
if dataset_path.exists():
    list_of_datasets = sorted((entry.name for entry in dataset_path.iterdir() if entry.name.isdigit()), key=lambda s: int(s))
' '.join(list_of_datasets)

In [ ]:
if list_of_datasets:
    [entry.name for entry in Path(dataset_path, list_of_datasets[0]).iterdir()]

In [ ]:
df_ref_data = {'det_id' : [], 'signal_mean' : [], 'signal_std' : []}
filename = 'raw-after-ff'
for det in list_of_ref_datasets:
    df_ref_data['det_id'].append(int(det))
    data = np.load(f'{interim_path}/{ref_dataset}/{det}/{filename}.npy')
    analysis_circle = Circle.from_json(f'{interim_path}/{ref_dataset}/{det}/analysis-circle.json')
    mean, std = get_mean_std(data, analysis_circle)
    df_ref_data[f'signal_mean'].append(mean)
    df_ref_data[f'signal_std'].append(std)
print("done")
df_ref = pd.DataFrame.from_dict(df_ref_data)
df_ref['dose_Gy_per_signal'] = ref_dose_Gy / df_ref['signal_mean']
df_ref.head()

In [ ]:
df_data = defaultdict(list)
df_data['det_id'] = []
df_data['timestamp'] = []
for filename in ('raw', 'raw-bg-image-removed', 'raw-aligned', 'raw-after-ff'):
    df_data[f'{filename}_signal_mean'] = []
    df_data[f'{filename}_signal_std'] = []
for det in list_of_datasets:
    df_data['det_id'].append(int(det))
    df_data['timestamp'].append(get_timestamp(f'{raw_path}/{dataset}/{det}/Pos0/metadata.txt'))

    analysis_circle = Circle.from_json(f'{interim_path}/{dataset}/{det}/analysis-circle.json')
    aligned_analysis_circle = Circle.from_json(f'{interim_path}/{dataset}/{det}/aligned-analysis-circle.json')

    for filename in ('raw', 'raw-bg-image-removed', 'raw-aligned', 'raw-after-ff'):
        data = np.load(Path(dataset_path, det, f'{filename}.npy'))
        circle = analysis_circle
        if 'aligned' in filename:
            circle = aligned_analysis_circle
        mean, std = get_mean_std(data, circle)
        df_data[f'{filename}_signal_mean'].append(mean)
        df_data[f'{filename}_signal_std'].append(std)
df = pd.DataFrame.from_dict(df_data)
df.head()

# Figure 1 - raw signal

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
ax.errorbar(df.det_id, df['raw_signal_mean'], yerr=df['raw_signal_std'], fmt='', linestyle='', marker='.', label='raw')
apply_defaults(ax)
ax.legend(loc='lower right')
if list_of_datasets:
    ax.set_xlim(int(min(list_of_datasets, key=lambda s:int(s)))-1, int(max(list_of_datasets, key=lambda s:int(s)))+1);
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_xlabel('detector id')
ax.set_ylabel('signal [a.u.]');

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
ax.errorbar(df.timestamp, df['raw_signal_mean'], yerr=df['raw_signal_std'], fmt='', linestyle='', marker='.', label=dataset, color="C0")
apply_grid(ax)
ax.legend(loc='lower left')
ax.set_xlabel('detector readout time')
ax.set_ylabel('signal [a.u.]');
ax.set_title(f'raw signal from circle with radius {analysis_circle.r} px')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=datetime.timezone(datetime.timedelta(hours=1))))

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(df.timestamp, df.det_id, linestyle='', marker='.', label=dataset, color="C0")
apply_grid(ax)
ax.legend(loc='upper left')
ax.set_xlabel('detector readout time')
ax.set_ylabel('detector id')
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=datetime.timezone(datetime.timedelta(hours=1))));

# Figure 2 - processing

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
for id, filename in enumerate(('raw-bg-image-removed', 'raw-after-ff', 'raw-aligned')):
    ax.errorbar(df.det_id+id/10, df[f'{filename}_signal_mean'], yerr=df[f'{filename}_signal_std'], fmt='', linestyle='', marker='.', label=filename)
apply_defaults(ax)
ax.legend()
if list_of_datasets:
    ax.set_xlim(int(min(list_of_datasets, key=lambda s:int(s)))-1, int(max(list_of_datasets, key=lambda s:int(s)))+1);
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_xlabel('detector id')
ax.set_ylabel('signal [a.u.]');

# Figure 3 - doses

In [ ]:
df['dose_Gy_uncorrected_mean'] = np.nan
df['dose_Gy_uncorrected_std'] = np.nan
for det in list_of_datasets:
    det_id = int(det)
    if det_id in df_ref.det_id.values:
        df.loc[df.det_id==det_id, 'dose_Gy_uncorrected_mean'] = df_ref.loc[df_ref.det_id==det_id, 'dose_Gy_per_signal'].values * df.loc[df.det_id==det_id, 'raw-after-ff_signal_mean'].values
        df.loc[df.det_id==det_id, 'dose_Gy_uncorrected_std'] = df_ref.loc[df_ref.det_id==det_id, 'dose_Gy_per_signal'].values * df.loc[df.det_id==det_id, 'raw-after-ff_signal_std'].values

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
ax.errorbar(df.det_id, df['dose_Gy_uncorrected_mean'], yerr=df['dose_Gy_uncorrected_std'], fmt='', linestyle='', marker='.', label='dose')
apply_defaults(ax)
ax.legend(loc=3)
if list_of_datasets:
    ax.set_xlim(int(min(list_of_datasets, key=lambda s:int(s)))-1, int(max(list_of_datasets, key=lambda s:int(s)))+1);
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_xlabel('detector id')
ax.set_ylabel('dose [Gy]');

# Figure 4 - 2D images

In [ ]:
det = 4

In [ ]:
try:
    vmax_data = max(np.nanpercentile(a=np.load(Path(dataset_path, f'{det}', f'{filename}.npy')), q=95) for filename in ('raw-bg-image-removed', 'raw-after-ff', 'raw-aligned'))
    print(f'vmax_data = {vmax_data}')

    vmax_ref_data = max(np.nanpercentile(a=np.load(f'{interim_path}/{ref_dataset}/{det}/{filename}.npy'), q=95) for filename in ('raw-bg-image-removed', 'raw-after-ff', 'raw-aligned'))
    print(f'vmax_ref_data = {vmax_ref_data}')

    detector_circle = Circle.from_json(f'{interim_path}/{dataset}/{det}lv/det-circle.json')
    aligned_det_circle = Circle.from_json(f'{interim_path}/{dataset}/{det}lv/aligned-det-circle.json')

    analysis_circle = Circle.from_json(f'{interim_path}/{dataset}/{det}/analysis-circle.json')
    aligned_analysis_circle = Circle.from_json(f'{interim_path}/{dataset}/{det}/aligned-analysis-circle.json')
except FileNotFoundError:
    print(f'no data for detector {det}')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16,10))

for col_id, filename in enumerate(('raw-bg-image-removed', 'raw-after-ff', 'raw-aligned')):
    try:
        data = np.load(Path(dataset_path, f'{det}', f'{filename}.npy'))
    except FileNotFoundError:
        print(f'no data for detector {det}')
        continue
    data_for_plotting = np.clip(data, a_min=None, a_max=np.nanpercentile(a=data, q=95))
    data_plot = axes[0][col_id].imshow(data_for_plotting, cmap='terrain', vmin=0, vmax=vmax_data);
    axes[0][col_id].set_title(f'{dataset} {filename}')

    ref_data = np.load(f'{interim_path}/{ref_dataset}/{det}/{filename}.npy')
    ref_data_for_plotting = np.clip(ref_data, a_min=None, a_max=np.nanpercentile(a=ref_data, q=95))
    ref_data_plot = axes[1][col_id].imshow(ref_data_for_plotting, cmap='terrain', vmin=0, vmax=vmax_ref_data);
    axes[1][col_id].set_title(f'{ref_dataset} {filename}')

    ax = axes[0][col_id] 
    if 'aligned' in filename:
        ax.add_artist(plt.Circle(xy=(aligned_det_circle.x, aligned_det_circle.y), radius=aligned_det_circle.r, color='black', fill=False, transform=ax.transData))
        ax.add_artist(plt.Circle(xy=(aligned_analysis_circle.x, aligned_analysis_circle.y), radius=aligned_analysis_circle.r, color='red', fill=False, transform=ax.transData))
    else:
        ax.add_artist(plt.Circle(xy=(detector_circle.x, detector_circle.y), radius=detector_circle.r, color='black', fill=False, transform=ax.transData))
        ax.add_artist(plt.Circle(xy=(analysis_circle.x, analysis_circle.y), radius=analysis_circle.r, color='red', fill=False, transform=ax.transData))

fig.colorbar(data_plot, ax=axes[0], location='right', shrink=0.75)
fig.colorbar(ref_data_plot, ax=axes[1], location='right', shrink=0.75);

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,5))
try:
    data = np.load(Path(dataset_path, f'{det}', f'raw-aligned.npy'))
    ref_data = np.load(f'{interim_path}/{ref_dataset}/{det}/raw-aligned.npy')
except  FileNotFoundError:
    print(f'no data for detector {det}')
    pass
data_Gy_uncorrected = data * ref_dose_Gy / ref_data
data_for_plotting = np.clip(data_Gy_uncorrected, a_min=None, a_max=np.nanpercentile(a=data_Gy_uncorrected, q=95))
data_plot = ax.imshow(data_for_plotting, cmap='terrain', vmin=0);
ax.add_artist(plt.Circle(xy=(aligned_det_circle.x, aligned_det_circle.y), radius=aligned_det_circle.r, color='black', fill=False, transform=ax.transData))
ax.add_artist(plt.Circle(xy=(aligned_analysis_circle.x, aligned_analysis_circle.y), radius=aligned_analysis_circle.r, color='red', fill=False, transform=ax.transData))
fig.colorbar(data_plot, ax=ax, location='right', shrink=0.75);